In [77]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil

# Mueve todos los fichero del directorio de trabajo a otro de archivo

In [78]:
source_dir = './BOEs'
target_dir = './BOEs_Anteriores'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    #shutil.move(os.path.join(source_dir, file_name), target_dir)
    # Evita que de error si el fichero que se mueve ya existe en dir destino
    try:
        os.remove(os.path.join(target_dir, file_name))
        shutil.move(os.path.join(source_dir, file_name), target_dir)
    except OSError:
        shutil.move(os.path.join(source_dir, file_name), target_dir)

# Recoge resumen diario del BOE de hoy

In [79]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210322
dia = 22
mes = 03
año = 2021
22/03/2021


In [80]:
URL_XML_resumen =  "https://www.boe.es/diario_boe/xml.php?id=BOE-S-" + str(hoy)

In [81]:
# URL_XML_resumen = 'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20210223' # Contiene Seccion T. Constitucional
# URL_XML_resumen = 'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20210305' # Contiene Sección V. A
# URL_XML_resumen = 'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20210216'

In [82]:
URL_XML_resumen

'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20210322'

In [83]:
url = URL_XML_resumen
r = requests.get(url)

In [84]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

save_html(r.content, './BOEs/Resumen-BOE-' + hoy + '.xml')

In [85]:
resumen = etree.parse('./BOEs/Resumen-BOE-' + hoy + '.xml')

In [86]:
raiz=resumen.getroot()
raiz.tag
raiz_sumario = raiz

In [87]:
seccion = raiz.findall("sumario/diario/seccion")
seccion

[]

In [88]:
for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')
    print(nombre_seccion)

['I. Disposiciones generales']
['II. Autoridades y personal. - A. Nombramientos, situaciones e incidencias']
['II. Autoridades y personal. - B. Oposiciones y concursos']
['III. Otras disposiciones']
['IV. Administración de Justicia']
['V. Anuncios. - A. Contratación del Sector Público']
['V. Anuncios. - B. Otros anuncios oficiales']
['V. Anuncios. - C. Anuncios particulares']
['T.C. Sección del Tribunal Constitucional']


In [89]:
secciones = raiz.findall("sumario/diario/seccion")
for seccion in secciones:
    print(seccion.text)

In [90]:
tabla_resumen = pd.DataFrame()

In [91]:
for secciones in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales") or contains(@nombre, "III. Otras disposiciones") or contains(@nombre, "V. Anuncios. - A. Contratación del Sector Público") or contains(@nombre, "T.C. Sección del Tribunal Constitucional")]'):

    nombre_seccion = secciones.xpath('@nombre')
    print(nombre_seccion)

    for seccion in secciones:

        for item in seccion.xpath('.//item'):
            item_id = item.xpath('@id')
            print(item_id)
            nombre_departamento = item.xpath('.//ancestor::departamento/@nombre')
            print(nombre_departamento)
            nombre_epigrafe = item.xpath('.//ancestor::epigrafe/@nombre')
            print(nombre_epigrafe)
            item_name = item.xpath('.//titulo/text()')
            item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]
            print(item_urlXml)
            tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                  'Departamento': nombre_departamento, 
                                                  'Epigrafe' : nombre_epigrafe,
                                                  'Item_id': item_id, 
                                                  'Item_Nombre' : item_name, 
                                                  'Item_URL_XML' : item_urlXml},
                                                  ignore_index=True)

['I. Disposiciones generales']
['BOE-A-2021-4396']
['MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL RETO DEMOGRÁFICO']
['Estado de alarma. Energía eléctrica']
https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-4396
['BOE-A-2021-4397']
['COMUNIDAD AUTÓNOMA DE CANARIAS']
['Escuelas infantiles']
https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-4397
['BOE-A-2021-4398']
['COMUNIDAD AUTÓNOMA DE CANARIAS']
['Infancia']
https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-4398
['BOE-A-2021-4399']
['COMUNIDAD AUTÓNOMA DE CANARIAS']
['Medidas extraordinarias']
https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-4399
['III. Otras disposiciones']
['BOE-A-2021-4469']
['MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA URBANA']
['Centro de Estudios y Experimentación de Obras Públicas. Cuentas anuales']
https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-4469
['BOE-A-2021-4470']
['MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA URBANA']
['Contratación administrativa']
https://www.boe.es/diario_boe/xml.php

In [92]:
len(tabla_resumen)

72

In [93]:
tabla_resumen['Item_id'][5:8]

5    [BOE-A-2021-4470]
6    [BOE-A-2021-4471]
7    [BOE-A-2021-4472]
Name: Item_id, dtype: object

In [94]:
tabla_resumen['Item_URL_XML'][5:8]

5    https://www.boe.es/diario_boe/xml.php?id=BOE-A...
6    https://www.boe.es/diario_boe/xml.php?id=BOE-A...
7    https://www.boe.es/diario_boe/xml.php?id=BOE-A...
Name: Item_URL_XML, dtype: object

In [95]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL ...,[Estado de alarma. Energía eléctrica],"[Orden TED/260/2021, de 18 de marzo, por la qu...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4396],[I. Disposiciones generales]
1,[COMUNIDAD AUTÓNOMA DE CANARIAS],[Escuelas infantiles],"[Decreto-ley 22/2020, de 23 de diciembre, de g...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4397],[I. Disposiciones generales]
2,[COMUNIDAD AUTÓNOMA DE CANARIAS],[Infancia],"[Decreto-ley 23/2020, de 23 de diciembre, por ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4398],[I. Disposiciones generales]
3,[COMUNIDAD AUTÓNOMA DE CANARIAS],[Medidas extraordinarias],"[Decreto-ley 24/2020, de 23 de diciembre, de m...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4399],[I. Disposiciones generales]
4,"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[Centro de Estudios y Experimentación de Obras...,"[Resolución de 2 de marzo de 2021, del Centro ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4469],[III. Otras disposiciones]
...,...,...,...,...,...,...
43,[MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL ...,[],[Anuncio de licitación de: Comité Ejecutivo de...,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-13870],[V. Anuncios. - A. Contratación del Sector Púb...
44,[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[],[Anuncio de formalización de contratos de: Ins...,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-13871],[V. Anuncios. - A. Contratación del Sector Púb...
45,"[MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y M...",[],[Anuncio de licitación de: Secretaría General ...,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-13872],[V. Anuncios. - A. Contratación del Sector Púb...
46,"[MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y M...",[],[Anuncio de licitación de: TGSS-Dirección prov...,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-13873],[V. Anuncios. - A. Contratación del Sector Púb...


In [96]:
len(tabla_resumen)

72

In [97]:
# Convierte columnas Departamento y Epigrafe a tipo str para poder realizar búsqueda de palabra Comunidad y variantes
tabla_resumen['Epigrafe'] = tabla_resumen['Epigrafe'].astype(str)
tabla_resumen['Departamento'] = tabla_resumen['Departamento'].astype(str)

In [98]:
# Elimina disposiciones con departamento o epigrafe que contenga las palabras Comunidad y sus variantes
terminos = ['Comunidad', 'Comunitat', 'COMUNIDAD', 'COMUNITAT']
tabla_resumen = tabla_resumen.drop(tabla_resumen.index[tabla_resumen['Epigrafe'].str.contains('|'.join(terminos), na=False)])
tabla_resumen = tabla_resumen.drop(tabla_resumen.index[tabla_resumen['Departamento'].str.contains('|'.join(terminos), na=False)])

In [99]:
len(tabla_resumen)

69

In [100]:
tabla_resumen[15:35]

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
18,['MINISTERIO DE POLÍTICA TERRITORIAL Y FUNCIÓN...,['Procedimientos de declaración de inconstituc...,"[Resolución de 3 de marzo de 2021, de la Secre...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4483],[III. Otras disposiciones]
19,['MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL...,['Sector gasista'],"[Resolución de 11 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4484],[III. Otras disposiciones]
20,['MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL...,['Sector gasista'],"[Resolución de 11 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4485],[III. Otras disposiciones]
21,['MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL...,['Sector gasista'],"[Resolución de 11 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4486],[III. Otras disposiciones]
22,['MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL...,['Sector gasista'],"[Resolución de 11 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4487],[III. Otras disposiciones]
23,['MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL...,['Sector gasista'],"[Resolución de 11 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4488],[III. Otras disposiciones]
24,['MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL...,['Sector gasista'],"[Resolución de 11 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4489],[III. Otras disposiciones]
25,['MINISTERIO DE DERECHOS SOCIALES Y AGENDA 2030'],['Premios'],"[Resolución de 10 de marzo de 2021, del Instit...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4490],[III. Otras disposiciones]
26,['MINISTERIO DE UNIVERSIDADES'],['Convenios'],"[Resolución de 11 de marzo de 2021, de la Agen...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4491],[III. Otras disposiciones]
27,['MINISTERIO DE JUSTICIA'],[],[Anuncio de formalización de contratos de: Sub...,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-13854],[V. Anuncios. - A. Contratación del Sector Púb...


In [101]:
tabla_resumen[tabla_resumen['Epigrafe'].str.contains('|'.join(terminos), na=False)]

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion


In [102]:
len(tabla_resumen)

69

In [103]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,['MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL...,['Estado de alarma. Energía eléctrica'],"[Orden TED/260/2021, de 18 de marzo, por la qu...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4396],[I. Disposiciones generales]
4,"['MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGEND...",['Centro de Estudios y Experimentación de Obra...,"[Resolución de 2 de marzo de 2021, del Centro ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4469],[III. Otras disposiciones]
5,"['MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGEND...",['Contratación administrativa'],"[Resolución de 23 de febrero de 2021, de la Au...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4470],[III. Otras disposiciones]
6,"['MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGEND...",['Encomienda de gestión'],"[Resolución de 3 de marzo de 2021, de la Agenc...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4471],[III. Otras disposiciones]
7,"['MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGEND...",['Premios'],"[Orden TMA/264/2021, de 3 de marzo, por la que...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-4472],[III. Otras disposiciones]
...,...,...,...,...,...,...
43,['MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL...,[],[Anuncio de licitación de: Comité Ejecutivo de...,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-13870],[V. Anuncios. - A. Contratación del Sector Púb...
44,['MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORM...,[],[Anuncio de formalización de contratos de: Ins...,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-13871],[V. Anuncios. - A. Contratación del Sector Púb...
45,"['MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y ...",[],[Anuncio de licitación de: Secretaría General ...,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-13872],[V. Anuncios. - A. Contratación del Sector Púb...
46,"['MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y ...",[],[Anuncio de licitación de: TGSS-Dirección prov...,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-13873],[V. Anuncios. - A. Contratación del Sector Púb...


# Descarga ficheros XML asociados

In [104]:
for item_URL in tabla_resumen['Item_URL_XML']:
    r = requests.get(item_URL)
    #f = './BOEs/' + item_URL[-16:] + '.xml'
    ### Separa el número del BOE del resto de la cadena y aplica expresión REGEX 
    print(item_URL.split('='))
    print(re.match('BOE\-(A|B)\-[0-9]+\-[0-9]+',item_URL.split('=')[1]))
    filename = re.match('BOE\-(A|B)\-[0-9]+\-[0-9]+',item_URL.split('=')[1]).group()
    print(filename)
    f = './BOEs/' + filename + '.xml'
    save_html(r.content, f)

['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-4396']
<re.Match object; span=(0, 15), match='BOE-A-2021-4396'>
BOE-A-2021-4396
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-4469']
<re.Match object; span=(0, 15), match='BOE-A-2021-4469'>
BOE-A-2021-4469
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-4470']
<re.Match object; span=(0, 15), match='BOE-A-2021-4470'>
BOE-A-2021-4470
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-4471']
<re.Match object; span=(0, 15), match='BOE-A-2021-4471'>
BOE-A-2021-4471
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-4472']
<re.Match object; span=(0, 15), match='BOE-A-2021-4472'>
BOE-A-2021-4472
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-4473']
<re.Match object; span=(0, 15), match='BOE-A-2021-4473'>
BOE-A-2021-4473
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-4474']
<re.Match object; span=(0, 15), match='BOE-A-2021-4474'>
BOE-A-2021-4474
['https://www.boe.es/diario_boe/xml.php?i

# Importa de nuevo los XML generados

In [105]:

import glob
BOEs = glob.glob('./BOEs/BOE*.xml')
print (BOEs)

['./BOEs/BOE-B-2021-13860.xml', './BOEs/BOE-B-2021-13874.xml', './BOEs/BOE-A-2021-4481.xml', './BOEs/BOE-A-2021-4495.xml', './BOEs/BOE-A-2021-4494.xml', './BOEs/BOE-A-2021-4480.xml', './BOEs/BOE-B-2021-13861.xml', './BOEs/BOE-A-2021-4509.xml', './BOEs/BOE-B-2021-13863.xml', './BOEs/BOE-A-2021-4469.xml', './BOEs/BOE-A-2021-4496.xml', './BOEs/BOE-A-2021-4482.xml', './BOEs/BOE-A-2021-4483.xml', './BOEs/BOE-A-2021-4497.xml', './BOEs/BOE-B-2021-13862.xml', './BOEs/BOE-A-2021-4508.xml', './BOEs/BOE-B-2021-13872.xml', './BOEs/BOE-B-2021-13866.xml', './BOEs/BOE-A-2021-4478.xml', './BOEs/BOE-A-2021-4493.xml', './BOEs/BOE-A-2021-4487.xml', './BOEs/BOE-A-2021-4486.xml', './BOEs/BOE-A-2021-4492.xml', './BOEs/BOE-A-2021-4479.xml', './BOEs/BOE-B-2021-13867.xml', './BOEs/BOE-B-2021-13873.xml', './BOEs/BOE-B-2021-13865.xml', './BOEs/BOE-B-2021-13871.xml', './BOEs/BOE-B-2021-13859.xml', './BOEs/BOE-A-2021-4484.xml', './BOEs/BOE-A-2021-4490.xml', './BOEs/BOE-A-2021-4491.xml', './BOEs/BOE-A-2021-4485.xml

# Genera DF con datos Análisis de cada XML

In [106]:
tabla_analisis = pd.DataFrame()

for BOE in BOEs:
    print (BOE)
    BOE_XML = etree.parse(BOE)
    raiz=BOE_XML.getroot()

    materias = [materia.text for materia in raiz.findall('analisis/materias/materia')]
    alertas = [alerta.text for alerta in raiz.findall('analisis/alertas/alerta')]
    
    referencias = [str(referencia.xpath('@referencia')).replace("['",'').replace("']",'') for referencia in raiz.findall('analisis/referencias/anteriores/anterior')]
    referencias_palabras = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/palabra')]
    referencias_texto = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/texto')]

    Item_Id = raiz.xpath(".//identificador/text()")
    Item_name = raiz.xpath('.//titulo/text()') 

    print(Item_Id)
    Fecha_Publicacion = datetime.strptime(str(raiz.xpath('.//fecha_publicacion/text()'))[2:-2],"%Y%m%d")
    print(Fecha_Publicacion)
    #Fecha_Publicacion = str(raiz.xpath('.//fecha_publicacion/text()'))[2:-2]
 
    tabla_analisis = tabla_analisis.append({'Item_id': Item_Id,
                                            'Item_Name' : Item_name,
                                            'Fecha_publicacion' : Fecha_Publicacion,
                                            'Materias' : materias,
                                            'Alertas' : alertas,
                                            'Referencias' : referencias,
                                            'Referencias_palabra' : referencias_palabras,
                                            'Referencias_texto' : referencias_texto},
                                            ignore_index=True)


./BOEs/BOE-B-2021-13860.xml
['BOE-B-2021-13860']
2021-03-22 00:00:00
./BOEs/BOE-B-2021-13874.xml
['BOE-B-2021-13874']
2021-03-22 00:00:00
./BOEs/BOE-A-2021-4481.xml
['BOE-A-2021-4481']
2021-03-22 00:00:00
./BOEs/BOE-A-2021-4495.xml
['BOE-A-2021-4495']
2021-03-22 00:00:00
./BOEs/BOE-A-2021-4494.xml
['BOE-A-2021-4494']
2021-03-22 00:00:00
./BOEs/BOE-A-2021-4480.xml
['BOE-A-2021-4480']
2021-03-22 00:00:00
./BOEs/BOE-B-2021-13861.xml
['BOE-B-2021-13861']
2021-03-22 00:00:00
./BOEs/BOE-A-2021-4509.xml
['BOE-A-2021-4509']
2021-03-22 00:00:00
./BOEs/BOE-B-2021-13863.xml
['BOE-B-2021-13863']
2021-03-22 00:00:00
./BOEs/BOE-A-2021-4469.xml
['BOE-A-2021-4469']
2021-03-22 00:00:00
./BOEs/BOE-A-2021-4496.xml
['BOE-A-2021-4496']
2021-03-22 00:00:00
./BOEs/BOE-A-2021-4482.xml
['BOE-A-2021-4482']
2021-03-22 00:00:00
./BOEs/BOE-A-2021-4483.xml
['BOE-A-2021-4483']
2021-03-22 00:00:00
./BOEs/BOE-A-2021-4497.xml
['BOE-A-2021-4497']
2021-03-22 00:00:00
./BOEs/BOE-B-2021-13862.xml
['BOE-B-2021-13862']
2021-

In [107]:
tabla_analisis.sort_values('Item_id')

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto
36,[Energía],2021-03-22,"[Orden TED/260/2021, de 18 de marzo, por la qu...",[BOE-A-2021-4396],"[Carburantes y combustibles, Derechos de conta...","[BOE-A-2020-6621, BOE-A-2014-6123, BOE-A-2020-...","[DE CONFORMIDAD con, DE CONFORMIDAD con, EN RE...","[el Real Decreto-ley 23/2020, de 23 de junio, ..."
9,[],2021-03-22,"[Resolución de 2 de marzo de 2021, del Centro ...",[BOE-A-2021-4469],[],[],[],[]
58,"[Derecho Administrativo, Organización de la Ad...",2021-03-22,"[Resolución de 23 de febrero de 2021, de la Au...",[BOE-A-2021-4470],"[Asturias, Autoridades Portuarias, Mesas de Co...",[],[],[]
57,[],2021-03-22,"[Resolución de 3 de marzo de 2021, de la Agenc...",[BOE-A-2021-4471],[],[],[],[]
63,[],2021-03-22,"[Orden TMA/264/2021, de 3 de marzo, por la que...",[BOE-A-2021-4472],[],[],[],[]
...,...,...,...,...,...,...,...,...
34,[],2021-03-22,[Anuncio de licitación de: Comité Ejecutivo de...,[BOE-B-2021-13870],[],[],[],[]
27,[],2021-03-22,[Anuncio de formalización de contratos de: Ins...,[BOE-B-2021-13871],[],[],[],[]
16,[],2021-03-22,[Anuncio de licitación de: Secretaría General ...,[BOE-B-2021-13872],[],[],[],[]
25,[],2021-03-22,[Anuncio de licitación de: TGSS-Dirección prov...,[BOE-B-2021-13873],[],[],[],[]


In [108]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
tabla_analisis['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
tabla_analisis['Tags'] = [[] for i in range(len(tabla_analisis))]
tabla_analisis['Match_ASECORP_BBDD'] = [[] for i in range(len(tabla_analisis))]


In [109]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
pattern = ['Ley [0-9]+\/[0-9]+','Ley Orgánica [0-9]+\/[0-9]+','Decreto [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Acuerdo Multilateral RID [0-9]+\/[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+', 'Instrucción IS\-[0-9]+']

In [110]:
# Consolida las columnas Referencias_palabra y Referencias_texto en una única frase
for i, row in tabla_analisis.iterrows():
    for item_list in range(len(row['Referencias'])): 
        row['Referencias_completas'].append(row['Referencias_palabra'][item_list] + ' ' + row['Referencias_texto'][item_list])
        tabla_analisis['Tags'][i] = re.findall('|'.join(pattern), str(row['Referencias_texto']), flags=re.IGNORECASE)
    print(row['Referencias_completas'])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['DE CONFORMIDAD con el Real Decreto-ley 23/2020, de 23 de junio', 'DE CONFORMIDAD con el Real Decreto 413/2014, de 6 de junio', 'EN RELACIÓN con la Orden TED/171/2020, de 24 de febrero', 'CITA Real Decreto 463/2020, de 14 de marzo']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
<ipython-input-110-6f892bcd60b0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_analisis['Tags'][i] = re.findall('|'.join(pattern), str(row['Referencias_texto']), flags=re.IGNORECASE)


In [111]:
# Elimina Tags duplicados
for i, row in tabla_analisis.iterrows():
    tabla_analisis['Tags'][i] = list(set(tabla_analisis['Tags'][i]))
    print(tabla_analisis['Tags'][i])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]<ipython-input-111-462252720182>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_analisis['Tags'][i] = list(set(tabla_analisis['Tags'][i]))

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Real Decreto-ley 23/2020', 'Orden TED/171/2020', 'Real Decreto 463/2020', 'Real Decreto 413/2014']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [112]:
tabla_analisis

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto,Referencias_completas,Tags,Match_ASECORP_BBDD
0,[],2021-03-22,[Anuncio de licitación de: ADIF Alta Velocidad...,[BOE-B-2021-13860],[],[],[],[],[],[],[]
1,[],2021-03-22,[Anuncio de formalización de contratos de: TGS...,[BOE-B-2021-13874],[],[],[],[],[],[],[]
2,[],2021-03-22,"[Resolución de 3 de marzo de 2021, de la Secre...",[BOE-A-2021-4481],[],[],[],[],[],[],[]
3,[],2021-03-22,"[Sala Segunda. Sentencia 20/2021, de 15 de feb...",[BOE-A-2021-4495],[],[],[],[],[],[],[]
4,[],2021-03-22,"[Sala Primera. Sentencia 19/2021, de 15 de feb...",[BOE-A-2021-4494],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
64,[],2021-03-22,"[Sala Primera. Sentencia 24/2021, de 15 de feb...",[BOE-A-2021-4499],[],[],[],[],[],[],[]
65,[],2021-03-22,"[Sala Primera. Sentencia 23/2021, de 15 de feb...",[BOE-A-2021-4498],[],[],[],[],[],[],[]
66,[],2021-03-22,"[Resolución de 15 de marzo de 2021, de la Secr...",[BOE-A-2021-4473],[],[],[],[],[],[],[]
67,[],2021-03-22,"[Sala Primera. Sentencia 32/2021, de 15 de feb...",[BOE-A-2021-4507],[],[],[],[],[],[],[]


In [113]:
texto = ''
for refs in tabla_analisis['Referencias_completas']:
    for ref in refs:
        texto = texto + ref + ' '
        print(ref)

DE CONFORMIDAD con el Real Decreto-ley 23/2020, de 23 de junio
DE CONFORMIDAD con el Real Decreto 413/2014, de 6 de junio
EN RELACIÓN con la Orden TED/171/2020, de 24 de febrero
CITA Real Decreto 463/2020, de 14 de marzo


In [114]:
texto = ''
for i, row in tabla_analisis.iterrows():
    #unique_id = i
    print('\n' + str(row['Item_id']) + str(row['Item_Name']) +'\n')

    antecedente = 1
    for ref in row['Referencias_completas']:
        texto = texto + ref + ' '
        print('\t' + 'Antecedente ' + str(antecedente) + ': ' + ref + ' ' + str(re.findall('|'.join(pattern), ref, flags=re.IGNORECASE)))
        antecedente += 1


['BOE-B-2021-13860']['Anuncio de licitación de: ADIF Alta Velocidad - Presidencia. Objeto: Servicio de reposición de cable de energía a 750 v en el tramo Alcover-Vilafranca de la línea de Alta Velocidad Madrid-Barcelona-Figueres. Expediente: 3.21/20810.0007.']


['BOE-B-2021-13874']['Anuncio de formalización de contratos de: TGSS-Dirección provincial de Lugo. Objeto: Suministro de 100.000 mascarillas FFP2 con certificado de la C.E., para el personal de la Seguridad Social (INSS/TGSS) de Lugo. Expediente: 27UC2/21T.']


['BOE-A-2021-4481']['Resolución de 3 de marzo de 2021, de la Secretaría General de Coordinación Territorial, por la que se publica el Acuerdo de la Comisión de Cooperación entre la Comunidad de Castilla y León y el Estado, en relación con la Ley 2/2020, de 24 de noviembre, de modificación del texto refundido de las normas legales vigentes en materia de condiciones de acceso y disfrute de la prestación esencial de renta garantizada de ciudadanía de Castilla y León, aprob

In [115]:
# Aplica expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
regex_result = re.findall('|'.join(pattern), texto, flags=re.IGNORECASE)

In [116]:
print(regex_result)

['Real Decreto-ley 23/2020', 'Real Decreto 413/2014', 'Orden TED/171/2020', 'Real Decreto 463/2020']


In [117]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [118]:
boletin_flat_list

['Real Decreto-ley 23/2020',
 'Orden TED/171/2020',
 'Real Decreto 463/2020',
 'Real Decreto 413/2014']

## Importa BBDD ASECORP

In [119]:
ASECORP_BBDD = pd.read_csv('./ASECORP/ExportNormes_20210126.csv', delimiter=';')

In [120]:
ASECORP_BBDD

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
0,RES SLT/2313/2020 CAT,2313,"Resolución SLT/2313/2020, de 24 de septiembre,...",CAT,Cataluña,NaN,Prevención,[2]
1,(zzz) RES 12/4/2007 AST,12/4/2007,"Resolución de 12 abril de 2007, de la Consejer...",AST,Asturias,NaN,M. Ambiente,[1]
2,A CORUÑA 10/7/2015,10/7/2015,Modificación de la Ordenanza de vertidos y ser...,GAL,Galicia,A Coruña,M. Ambiente,[1]
3,A CORUÑA 11/8/2014 (1),11/8/2014,Ordenanza municipal reguladora de los procedim...,GAL,Galicia,A Coruña,M. Ambiente,[1]
4,A CORUÑA 11/8/2014 (2),11/8/2014,Ordenanza de protección contra la contaminació...,GAL,Galicia,A Coruña,M. Ambiente,[1]
...,...,...,...,...,...,...,...,...
19152,ZUMAIA 1/3/2001,1/3/2001,Ordenanza reguladora de la gestión del ciclo i...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19153,ZUMAIA 12/11/1999,12/11/1999,Normativa complementaria de las Normas Subsidi...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19154,ZUMAIA 15/6/2012,15/6/2012,Modificación de la Ordenanza municipal de la g...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19155,ZUMAIA 6/2/2004,6/2/2004,Modificación de la Normativa complementaria de...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]


In [121]:
ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'] == 'España']

In [122]:
ASECORP_BBDD_BOE

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
23,ACU 1/2020 ESP,1,Acuerdo Multilateral RID 1/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
35,ACU 11/6/2007 ESP,11/6/2007,Entrada en vigor del Acuerdo entre el Reino de...,ESP,España,NaN,M. Ambiente,[1]
57,ACU 17/10/2006 ESP,17/10/2006,Aplicación provisional del Acuerdo entre el Re...,ESP,España,NaN,M. Ambiente,[1]
71,ACU 19/11/2018 ESP,19/11/2018,Acuerdo de la Junta de Gobierno de la Confeder...,ESP,España,NaN,M. Ambiente,[1]
82,ACU 2/2020 ESP,2,Acuerdo Multilateral RID 2/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
...,...,...,...,...,...,...,...,...
18496,STC 86/2019 ESP,86,"Sentencia 86/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18497,STC 87/2019 ESP,87,"Sentencia 87/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18498,STC 88/2018 ESP,88,"Sentencia 88/2018, de 19 de julio de 2018. Con...",ESP,España,NaN,M. Ambiente / Reg. Industrial,[1][3]
18499,STC 9/3/2010 ESP,9/3/2010,"Sentencia de 9 de marzo de 2010, de la Sala Te...",ESP,España,NaN,M. Ambiente,[1]


In [123]:
# Crea nueva columna vacía de tipo lista en ASECORP_BBDD
ASECORP_BBDD_BOE['Tags'] = [[] for i in range(len(ASECORP_BBDD_BOE))]

<ipython-input-123-34866c974db0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ASECORP_BBDD_BOE['Tags'] = [[] for i in range(len(ASECORP_BBDD_BOE))]


In [124]:
titulo = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    titulo.append(str(row['Titulo']) + str(re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)))
    ASECORP_BBDD_BOE['Tags'][i] = re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)

/Users/Alfredo/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [125]:
ASECORP_BBDD_BOE[1200:1250]

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma,Tags
11629,ORD PRE/2827/2009 ESP,2827,"Orden PRE/2827/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/2827/2009, Real Decreto 1370/2006]"
11630,ORD PRE/2843/2009 ESP,2843,"Orden PRE/2843/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2843/2009, Real Decreto 2163/1994]"
11631,ORD PRE/2851/2010 ESP,2851,"Orden PRE/2851/2010, de 4 de noviembre, por la...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2851/2010, Real Decreto 2163/1994]"
11632,ORD PRE/2871/2011 ESP,2871,"Orden PRE/2871/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2871/2011, Real Decreto 2163/1994]"
11633,ORD PRE/2872/2011 ESP,2872,"Orden PRE/2872/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2872/2011, Real Decreto 2163/1994]"
11634,ORD PRE/29/2004 ESP,29,"Orden PRE/29/2004, de 15 de enero, por la que ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/29/2004, Real Decreto 837/2002]"
11635,ORD PRE/2922/2005 ESP,2922,"Orden PRE/2922/2005, de 19 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2922/2005, Orden de 8 de marzo de 1..."
11636,ORD PRE/2957/2008 ESP,2957,"Orden PRE/2957/2008, de 10 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2957/2008, Real Decreto 2163/1994]"
11637,ORD PRE/3/2006 ESP,3,"Orden PRE/3/2006, de 12 de enero, por la que s...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3/2006, Real Decreto 255/2003]"
11638,ORD PRE/3159/2004 ESP,3159,"Orden PRE/3159/2004, de 28 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3159/2004, Real Decreto 1406/1989]"


In [126]:
# titulo

In [127]:
len(titulo)

2959

In [128]:
## Busca patrones regex definidos en columna títulos del DF 
boletin = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    boletin.append(str(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)))

In [129]:
boletin[0:25]

["['Acuerdo Multilateral RID 1/2020']",
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral RID 2/2020']",
 '[]',
 "['Acuerdo Multilateral RID 1/2013']",
 '[]',
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral RID 3/2007']",
 "['Acuerdo Multilateral RID 4/2009']",
 '[]',
 '[]',
 "['Acuerdo Multilateral M-150']",
 "['Acuerdo Multilateral M-159']",
 "['Acuerdo multilateral M-168']",
 "['Acuerdo Multilateral M-170']",
 "['Acuerdo Multilateral M-171']",
 "['Acuerdo Multilateral M-173']",
 "['Acuerdo Multilateral M-175']",
 '[]',
 "['Acuerdo Multilateral M-178']",
 "['Acuerdo Multilateral M-198']"]

In [130]:
## Para cada fila de la BBDD recoge la expresión REGEX encontrada y si no existe no la incluye 
## en la lista resultante llamada 'boletin'. Además si no detecta expresión la cuenta como vacía
## en variable 'n_vacios', y la añade a la lista 'vacios' para inspeccionar posteriormente 
boletin = []
vacios = []
n_vacios = 0
for i, row in ASECORP_BBDD_BOE.iterrows():
    regex_result = re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)
    if len(regex_result) != 0:
        boletin.append(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE))
    else:
        n_vacios += 1
        vacios.append(row['Titulo'])

In [131]:
n_vacios

225

In [132]:
vacios[0:25]

['Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Aplicación provisional del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Acuerdo de la Junta de Gobierno de la Confederación Hidrográfica del Segura celebrada el día 30 de octubre de 2018 sobre aplicación de indisponibilidad o reducción de dotaciones de acuerdo al Plan Especial de Sequía y al Real Decreto que la regula',
 'Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre el traslado de residuos, hecho en Madrid el 29 de noviembre de 2011.',
 'Canje de Notas por el que se procede a la prórroga del mantenimiento en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 27 de enero de 2000.',
 'Aplicación provisional del

In [133]:
boletin[0:25]

[['Acuerdo Multilateral RID 1/2020'],
 ['Acuerdo Multilateral RID 2/2020'],
 ['Acuerdo Multilateral RID 1/2013'],
 ['Acuerdo Multilateral RID 3/2007'],
 ['Acuerdo Multilateral RID 4/2009'],
 ['Acuerdo Multilateral M-150'],
 ['Acuerdo Multilateral M-159'],
 ['Acuerdo multilateral M-168'],
 ['Acuerdo Multilateral M-170'],
 ['Acuerdo Multilateral M-171'],
 ['Acuerdo Multilateral M-173'],
 ['Acuerdo Multilateral M-175'],
 ['Acuerdo Multilateral M-178'],
 ['Acuerdo Multilateral M-198'],
 ['Acuerdo Multilateral M-214'],
 ['Acuerdo Multilateral M-218'],
 ['Acuerdo Multilateral M-226'],
 ['Acuerdo Multilateral M-228'],
 ['Acuerdo Multilateral M-231'],
 ['Acuerdo Multilateral M-236'],
 ['Acuerdo Multilateral M-245'],
 ['Acuerdo Multilateral M-254'],
 ['Acuerdo Multilateral M-259'],
 ['Acuerdo Multilateral M-265'],
 ['Acuerdo Multilateral M-268']]

In [134]:
# Flatten list of lists
boletin_ASECORP_flat_list = [item for sublist in boletin for item in sublist]

In [135]:
boletin_ASECORP_flat_list[0:25]

['Acuerdo Multilateral RID 1/2020',
 'Acuerdo Multilateral RID 2/2020',
 'Acuerdo Multilateral RID 1/2013',
 'Acuerdo Multilateral RID 3/2007',
 'Acuerdo Multilateral RID 4/2009',
 'Acuerdo Multilateral M-150',
 'Acuerdo Multilateral M-159',
 'Acuerdo multilateral M-168',
 'Acuerdo Multilateral M-170',
 'Acuerdo Multilateral M-171',
 'Acuerdo Multilateral M-173',
 'Acuerdo Multilateral M-175',
 'Acuerdo Multilateral M-178',
 'Acuerdo Multilateral M-198',
 'Acuerdo Multilateral M-214',
 'Acuerdo Multilateral M-218',
 'Acuerdo Multilateral M-226',
 'Acuerdo Multilateral M-228',
 'Acuerdo Multilateral M-231',
 'Acuerdo Multilateral M-236',
 'Acuerdo Multilateral M-245',
 'Acuerdo Multilateral M-254',
 'Acuerdo Multilateral M-259',
 'Acuerdo Multilateral M-265',
 'Acuerdo Multilateral M-268']

In [136]:
## Elimina duplicados
boletin_ASECORP_flat_list = list(set(boletin_ASECORP_flat_list))

In [137]:
boletin_ASECORP_flat_list[0:25]

['Orden JUS/992/2015',
 'Real Decreto 299/2019',
 'Real Decreto 3349/1983',
 'Real Decreto 1295/2003',
 'Resolución de 31 de julio de 2014',
 'Sentencia 70/2018',
 'Real Decreto 1010/2015',
 'Real Decreto 653/2003',
 'Resolución de 24 de noviembre de 2017',
 'Real Decreto 314/2016',
 'Real Decreto 731/2020',
 'Orden de 12 de marzo de 1982',
 'Orden IET/1882/2014',
 'Real Decreto 1564/2010',
 'Real Decreto 27/2021',
 'Real Decreto 1123/2000',
 'Resolución de 27 de enero de 2017',
 'Ley 17/2015',
 'Resolución de 31 de enero de 2013',
 'Real Decreto 1616/2009',
 'Orden de 3 de febrero de 1986',
 'Orden FOM/2931/2015',
 'Sentencia de 25 de octubre de 2017',
 'Orden AAA/1903/2013',
 'Real Decreto 1466/1995']

In [138]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

{'Real Decreto 413/2014', 'Real Decreto 463/2020', 'Real Decreto-ley 23/2020'}

In [139]:
#tabla_analisis['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in tabla_analisis['Tags']:
    for row_comparing in ASECORP_BBDD_BOE['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Real Decreto 413/2014'}
{'Real Decreto 463/2020'}
{'Real Decreto 463/2020'}
{'Real Decreto 413/2014'}
{'Real Decreto-ley 23/2020'}
{'Real Decreto 413/2014'}
{'Real Decreto 413/2014'}
{'Real Decreto 413/2014'}
{'Real Decreto 413/2014'}
{'Real Decreto 413/2014'}
{'Real Decreto 413/2014'}
{'Real Decreto 413/2014'}
{'Real Decreto 413/2014'}
{'Real Decreto 413/2014'}
{'Real Decreto 413/2014'}


In [140]:
for i, row_to_compare in tabla_analisis.iterrows():
    for j, row_comparing in ASECORP_BBDD_BOE.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            tabla_analisis['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD_BOE['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Real Decreto 413/2014'} ORD IET/1168/2014 ESP
{'Real Decreto 463/2020'} ORD INT/262/2020 ESP
{'Real Decreto 463/2020'} ORD INT/284/2020 ESP
{'Real Decreto 413/2014'} RD 413/2014 ESP
{'Real Decreto-ley 23/2020'} RDL 23/2020 ESP
{'Real Decreto 413/2014'} RES 18/12/2015 ESP (1)
{'Real Decreto 413/2014'} RES 9/2/2016 ESP
{'Real Decreto 413/2014'} STC 16/3/2017 ESP
{'Real Decreto 413/2014'} STC 19/9/2016 ESP
{'Real Decreto 413/2014'} STC 20/12/2017 ESP
{'Real Decreto 413/2014'} STC 20/6/2016 ESP (1)
{'Real Decreto 413/2014'} STC 24/4/2017 ESP
{'Real Decreto 413/2014'} STC 26/7/2016 ESP
{'Real Decreto 413/2014'} STC 4/5/2017 ESP
{'Real Decreto 413/2014'} STC 5/5/2017 ESP


In [141]:
tabla_analisis

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto,Referencias_completas,Tags,Match_ASECORP_BBDD
0,[],2021-03-22,[Anuncio de licitación de: ADIF Alta Velocidad...,[BOE-B-2021-13860],[],[],[],[],[],[],[]
1,[],2021-03-22,[Anuncio de formalización de contratos de: TGS...,[BOE-B-2021-13874],[],[],[],[],[],[],[]
2,[],2021-03-22,"[Resolución de 3 de marzo de 2021, de la Secre...",[BOE-A-2021-4481],[],[],[],[],[],[],[]
3,[],2021-03-22,"[Sala Segunda. Sentencia 20/2021, de 15 de feb...",[BOE-A-2021-4495],[],[],[],[],[],[],[]
4,[],2021-03-22,"[Sala Primera. Sentencia 19/2021, de 15 de feb...",[BOE-A-2021-4494],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
64,[],2021-03-22,"[Sala Primera. Sentencia 24/2021, de 15 de feb...",[BOE-A-2021-4499],[],[],[],[],[],[],[]
65,[],2021-03-22,"[Sala Primera. Sentencia 23/2021, de 15 de feb...",[BOE-A-2021-4498],[],[],[],[],[],[],[]
66,[],2021-03-22,"[Resolución de 15 de marzo de 2021, de la Secr...",[BOE-A-2021-4473],[],[],[],[],[],[],[]
67,[],2021-03-22,"[Sala Primera. Sentencia 32/2021, de 15 de feb...",[BOE-A-2021-4507],[],[],[],[],[],[],[]


In [142]:
tabla_analisis['Match_ASECORP_BBDD'][1]

[]

In [143]:
tabla_analisis['Tags'][1]

[]

# Genera Fichero EXCEL de resultados

In [144]:
## Cambia orden de columnas y elimina las no necesarias  
tabla_analisis_final = tabla_analisis[['Item_id','Item_Name','Fecha_publicacion','Materias','Alertas','Referencias','Referencias_completas','Tags','Match_ASECORP_BBDD']]

In [145]:
## Crea función que convierte lista a string en todas las columnas de tabla_analisis
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [146]:
## Aplica función de conversión de listas a strings
tabla_analisis_final = tabla_analisis_final.apply(lambda x: [list2Str(i) for i in x])

In [147]:
  tabla_analisis_final[0:10]

,Item_id,Item_Name,Fecha_publicacion,Materias,Alertas,Referencias,Referencias_completas,Tags,Match_ASECORP_BBDD
0,BOE-B-2021-13860,Anuncio de licitación de: ADIF Alta Velocidad ...,2021-03-22,,,,,,
1,BOE-B-2021-13874,Anuncio de formalización de contratos de: TGSS...,2021-03-22,,,,,,
2,BOE-A-2021-4481,"Resolución de 3 de marzo de 2021, de la Secret...",2021-03-22,,,,,,
3,BOE-A-2021-4495,"Sala Segunda. Sentencia 20/2021, de 15 de febr...",2021-03-22,,,,,,
4,BOE-A-2021-4494,"Sala Primera. Sentencia 19/2021, de 15 de febr...",2021-03-22,,,,,,
5,BOE-A-2021-4480,"Resolución de 3 de marzo de 2021, de la Secret...",2021-03-22,,,,,,
6,BOE-B-2021-13861,Anuncio de licitación de: ADIF - Presidencia. ...,2021-03-22,,,,,,
7,BOE-A-2021-4509,"Pleno. Sentencia 34/2021, de 17 de febrero de ...",2021-03-22,,,,,,
8,BOE-B-2021-13863,Anuncio de licitación de: Comité Ejecutivo de ...,2021-03-22,,,,,,
9,BOE-A-2021-4469,"Resolución de 2 de marzo de 2021, del Centro d...",2021-03-22,,,,,,


In [148]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

tabla_analisis_final_CSV = tabla_analisis_final

In [149]:
# Compone y genera enlace a PDF del BOE correspondiente
tabla_analisis_final_CSV['Item_id'] = '=HIPERVINCULO(' + '"https://www.boe.es/boe/dias/' \
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%Y')) + '/'\
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%m')) + '/'\
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%d')) + '/'\
                                                        + 'pdfs/'\
                                                        + tabla_analisis_final_CSV['Item_id'] + '.pdf";'\
                                                        + '"' + tabla_analisis_final_CSV['Item_id'] + '")'

In [150]:
print(tabla_analisis_final_CSV['Item_id'][0])

=HIPERVINCULO("https://www.boe.es/boe/dias/2021/03/22/pdfs/BOE-B-2021-13860.pdf";"BOE-B-2021-13860")


In [151]:
# tabla_analisis_final_CSV.to_excel("./ASECORP/Resultados_Matching.xlsx", sheet_name='MATCHING_BOE') 

In [152]:
tabla_analisis_final_CSV.to_csv("./ASECORP/Resultados_Matching_BOE_" + today.strftime("%Y%m%d") + ".csv", index=False) 

# ------------------------------------------------------------------------